In [2]:
from bs4 import BeautifulSoup as bs

In [17]:
doc_name = 'sites.html'

with open(doc_name) as fp:
    soup = bs(fp, 'html.parser')
# print(soup.prettify())

In [16]:
valid_links = [link for link in soup.find_all('a') if not 'bible-quiz-index' in str(link)]
for link in valid_links:
    print(link.get('href'))

http://www.religionresourcesonline.org/bible-quiz/print-52-women-in-the-old-testament-bible-quiz
http://www.religionresourcesonline.org/bible-quiz/answers-52-women-in-the-old-testament-bible-quiz
http://www.religionresourcesonline.org/bible-quiz/print-33-bible-quiz-who-said-that-new-testament-2
http://www.religionresourcesonline.org/bible-quiz/answers-33-bible-quiz-who-said-that-new-testament-2
http://www.religionresourcesonline.org/bible-quiz/print-3-bible-quiz-the-creation
http://www.religionresourcesonline.org/bible-quiz/answers-3-bible-quiz-the-creation
http://www.religionresourcesonline.org/bible-quiz/print-9-bible-quiz-noah-and-the-ark
http://www.religionresourcesonline.org/bible-quiz/answers-9-bible-quiz-noah-and-the-ark
http://www.religionresourcesonline.org/bible-quiz/print-302-bible-quiz-on-luke
http://www.religionresourcesonline.org/bible-quiz/answers-302-bible-quiz-on-luke
http://www.religionresourcesonline.org/bible-quiz/print-17-bible-quiz-moses-in-egypt
http://www.religi